In [1]:
import numpy as np
from scipy import stats
import scipy
from numpy import linalg

In [2]:
data=np.loadtxt(open('x.csv','rb'),delimiter=',')

In [3]:
#定义贝叶斯类
class bayes():
    def __init__(self,k,mean,a,b,v,W,samp):
        self.k=k
        self.samp=samp
        self.m_0=mean#row vector
        self.a_0=a#scalar
        self.b_0=b#scalar
        self.v_0=v#scalar
        self.W_0=W #matrix
        self.num_samp=self.samp.shape[0]
        self.dim_samp=self.samp.shape[1]
        self.W_samp=np.zeros([self.k,self.dim_samp,self.dim_samp])
        self.v_samp=np.ones([self.k])*self.v_0
        self.a_samp=np.ones([self.k])*self.a_0
        self.b_samp=np.ones([self.k])*self.b_0
        self.W_samp[:,:,:]=self.W_0
        self.mean_mat=np.random.rand(k,self.dim_samp)
        #self.mean_mat=np.zeros(k,self.dim_samp)
        for i_num in range(self.k):
            index_samp=np.random.randint(0,self.num_samp-1)
            self.mean_mat[i_num,:]=data[index_samp,:]
        #self.mean_mat[:,:]=self.m_0
        
    def post(self):
        D_dim=range(self.dim_samp)
        digam=np.array(D_dim)
        digam=digam+1
        post_mat=np.zeros([self.num_samp,self.k])
        temp_mat=np.zeros([self.num_samp,self.k])
        for i_k in range(self.k):
            var_m=self.samp-self.mean_mat[i_k,:]
            for_diag=np.dot(np.dot(var_m,self.W_samp[i_k,:,:]),var_m.T)
            test=np.diag(for_diag)
            #print(test)
            diag_mat=np.diag(for_diag)#.reshape(1,-1)
            #print(diag_mat)
            temp_muk=self.dim_samp*(1/self.b_samp[i_k])+self.v_samp[i_k]*diag_mat
            temp_muk=-1*temp_muk#.T
            ln_gama=scipy.special.digamma((self.v_samp[i_k]+1-digam)/2).sum()+self.dim_samp*np.log(2)+np.log(linalg.det(self.W_samp[i_k,:,:]))
            ln_pik=scipy.special.digamma(self.a_samp[i_k]-scipy.special.digamma(self.a_samp.sum()))
            ln_rho=ln_pik+0.5*ln_gama-((self.dim_samp)/2)*np.log(2*np.pi)+0.5*temp_muk#列向量
            temp_mat[:,i_k]=ln_rho
        #print(temp_mat)
        temp_rho=np.exp(temp_mat)
        sum_post=temp_rho.sum(axis=1).reshape(1,-1).T
        post_mat=temp_rho/sum_post
        print(temp_mat.sum())
        return post_mat
        
    def update_paras(self,post_mat):
        N_k=np.zeros([self.k])
        N_k=post_mat.sum(axis=0)
        mean=np.zeros([self.k,self.dim_samp])
        sigma=np.zeros([self.k,self.dim_samp,self.dim_samp])
        for i_k in range(self.k):
            weights=np.diag(post_mat[:,i_k])
            temp_mu=np.dot(weights,self.samp).sum(axis=0)
            mean[i_k,:]=temp_mu/N_k[i_k]
        
        for i_k in range(self.k):
            weights=np.diag(post_mat[:,i_k])
            temp_sigma=np.dot(np.dot((self.samp-mean[i_k,:]).T,weights),self.samp-mean[i_k,:])
            sigma[i_k,:,:]=temp_sigma/N_k[i_k]
            
        x_mean=mean
        x_S=sigma
        
        self.a_samp=self.a_0+N_k
        self.b_samp=self.b_0+N_k
        temp_m=np.zeros([self.k,self.dim_samp])
        for i_k in range(self.k):
            temp_m[i_k,:]=(1/self.b_samp[i_k])*((self.b_0*self.m_0)+N_k[i_k]*x_mean[i_k,:])
            
        self.mean_mat=temp_m
        
        
        temp_W=np.zeros([self.k,self.dim_samp,self.dim_samp])
        for i_k in range(self.k):
            
            #print((x_mean[i_k]-self.m_0).reshape(1,-1).T*(x_mean[i_k]-self.m_0))
            temp_Winv=linalg.inv(self.W_0)+N_k[i_k]*x_S[i_k,:,:]+((self.b_0*N_k[i_k])/(self.b_0+N_k[i_k]))*((x_mean[i_k]-self.m_0).reshape(1,-1).T*(x_mean[i_k]-self.m_0))
            temp_W[i_k,:,:]=linalg.inv(temp_Winv)
        #print(temp_W)
        self.W_samp=temp_W
        self.v_samp=self.v_0+N_k
        return temp_W,temp_m,self.v_samp,self.a_samp,self.b_samp
    
        
        

In [4]:
k=6 #the number of compoents, you can change it as your wish
train_time=100 #the number of iterations
m_0=np.array([3,3,3])#data[0,:]
a0=0.001
b0=5
v0=5
W0=np.array([[1,0,0],[0,1,0],[0,0,1]])
bayes_1=bayes(k,m_0,a0,b0,v0,W0,data)
for iters in range(train_time):
    print('the likelihood in the iteration '+str(iters+1)+' is ')
    post_mat=bayes_1.post()
    #print(post_mat)
    W_r,m_r,v_r,a_r,b_r=bayes_1.update_paras(post_mat)
    
print(m_r)
np.savetxt('z_VB.csv',post_mat,delimiter=',')
np.savetxt('params1_VB.dat',m_r,delimiter=',')
#out_sigma=np.vstack([sigma_r[0,:,:],sigma_r[1,:,:],sigma_r[2,:,:]])
#np.savetxt('params2.dat',out_sigma,delimiter=',')

the likelihood in the iteration 1 is 
-9413912.475058608
the likelihood in the iteration 2 is 
-1391194.0485399503
the likelihood in the iteration 3 is 
-1286789.2831538634
the likelihood in the iteration 4 is 
-1240331.417109836
the likelihood in the iteration 5 is 
-1210656.3201402356
the likelihood in the iteration 6 is 
-1172411.200401718
the likelihood in the iteration 7 is 
-1145152.8752044407
the likelihood in the iteration 8 is 
-1141005.5346650027
the likelihood in the iteration 9 is 
-1189113.0453278031
the likelihood in the iteration 10 is 
-1296901.6863788744
the likelihood in the iteration 11 is 
-1312291.7595482913
the likelihood in the iteration 12 is 
-1311344.2297694087
the likelihood in the iteration 13 is 
-1310976.3992010702
the likelihood in the iteration 14 is 
-1311093.5974507695
the likelihood in the iteration 15 is 
-1311537.0926707108
the likelihood in the iteration 16 is 
-1312232.079439349
the likelihood in the iteration 17 is 
-1313148.8335700708
the likeli

In [21]:
#draft, and ignore them!

[[0.92278434 3.11359759]
 [1.25611767 2.97052399]]
-31.999999999999986
[1. 1. 1.]
[0.92278434 2.97052399]
[[0.92278434 0.92278434]
 [2.97052399 2.97052399]]
[[0.92278434 2.97052399]
 [0.92278434 2.97052399]]
[[ 9.51628683 29.50185129]
 [10.51176117 26.50213589]]
[ 0.42278434  0.03648997 -1.96351003]
